In [1]:
from lm_nav.navigation_graph import NavigationGraph
from lm_nav.utils import rectify_and_crop
from pathlib import Path
import numpy as np
import io
from PIL import Image
import random
from tqdm import tqdm
import clip
from torchvision.transforms.functional import to_pil_image
import PIL
import torch

/home/chashi/.local/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Saving Graph

In [2]:
def save_images(graph: NavigationGraph, output_dir: str):
    output_dir_raw = Path(output_dir+'/raw')
    output_dir_recitified_cropped = Path(output_dir+'/recitified_cropped')

    output_dir_raw.mkdir(parents=True, exist_ok=True)
    output_dir_recitified_cropped.mkdir(parents=True, exist_ok=True)
    for i in range(graph.vert_count):
        for j, img_bytes in enumerate(graph._images[i]):
            # Save raw image
            with open(output_dir_raw / f'image_{i}_{j}_raw', 'wb') as f:
                f.write(img_bytes)

            # Save rectified and cropped image
            rectified_and_cropped = rectify_and_crop(np.array(Image.open(io.BytesIO(img_bytes))))
            #print(rectified_and_cropped.size)
            #rectified_and_cropped_image = Image.fromarray(rectified_and_cropped)
            rectified_and_cropped.save(output_dir_recitified_cropped / f'image_{i}_{j}_rectified_and_cropped.png')



In [2]:
graph = NavigationGraph("graphs/small_graph.pkl")

In [6]:
def preprocess_image(image_data, preprocess, device):
    """
    Loads and preprocesses an image from a given image data.
    Args:
        image_data (bytes): Image Data.
        preprocess (function): Preprocessing function (e.g., from CLIP).
        device (torch.device): Device to load the image onto.
    Returns:
        torch.Tensor: Preprocessed image tensor.
    """
    #image = Image.open(io.BytesIO(image_data))
    return preprocess(image_data).unsqueeze(0).to(device)

In [10]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)
model.eval()
landmarks = ['a stop sign']
text_labels = ["A photo of " + desc for desc in landmarks]
text_tokens = clip.tokenize(text_labels).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()
text_features /= text_features.norm(dim=-1, keepdim=True)
std_value, mean_value = torch.std_mean(text_features)
print('Text:', std_value, mean_value)
image_data = graph._images[77][0]
image_input = preprocess_image(PIL.Image.fromarray(np.array(PIL.Image.open(io.BytesIO(image_data)))), preprocess, device)
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
image_features /= image_features.norm(dim=-1, keepdim=True)

std_value, mean_value = torch.std_mean(image_features)
print('Image:', std_value, mean_value)

Text: tensor(0.0361, device='cuda:1') tensor(0.0010, device='cuda:1')
Image: tensor(0.0361, device='cuda:1') tensor(0.0004, device='cuda:1')


In [ ]:
save_images(graph, 'graph_test_images')

# Random Pixel Modification

In [4]:
from torchvision import transforms
def inverse_normalize():
    """
    Returns an inverse normalization transformation.
    Returns:
        transforms.Normalize: Transformation to apply inverse normalization.
    """
    mean = [0.48145466, 0.4578275, 0.40821073]
    std = [0.26862954, 0.26130258, 0.27577711]
    return transforms.Normalize(mean=[-m/s for m, s in zip(mean, std)], std=[1/s for s in std])


In [3]:


def random_blacken_image(image_array, percentage=10):
    """
    Randomly blackens a given percentage of pixels in the image array.

    Parameters:
    - image_array: numpy array of the image.
    - percentage: The percentage of pixels to blacken.

    Returns:
    - Modified image array with random pixels set to black.
    """
    # Calculate the total number of pixels to blacken
    total_pixels = image_array.shape[0] * image_array.shape[1]
    pixels_to_blacken = total_pixels * percentage // 100
    
    for _ in range(pixels_to_blacken):
        # Randomly select a pixel
        x = random.randint(0, image_array.shape[0] - 1)
        y = random.randint(0, image_array.shape[1] - 1)
        # Blacken the selected pixel
        image_array[x, y] = 0
    return image_array

def modify_and_save_images(graph):
    for i in tqdm(range(graph.vert_count)):
        for j, img_bytes in enumerate(graph._images[i]):
            # Load image
            img = Image.open(io.BytesIO(img_bytes))
            img_array = np.array(img)
            # Modify image by randomly blackening pixels
            modified_img_array = random_blacken_image(img_array, percentage=70)  # You can change percentage as needed
            modified_img = Image.fromarray(modified_img_array)
            # Update the graph's _images list with the modified image
            with io.BytesIO() as output:
                modified_img.save(output, format='PNG')
                graph._images[i][j] = output.getvalue()
    return graph

def recity_crop_modification(graph):
    for i in tqdm(range(graph.vert_count)):
        for j, img_bytes in enumerate(graph._images[i]):
            # Load image
            modified_img = rectify_and_crop(np.array(Image.open(io.BytesIO(img_bytes))))
            # Update the graph's _images list with the modified image
            with io.BytesIO() as output:
                modified_img.save(output, format='PNG')
                graph._images[i][j] = output.getvalue()
    return graph
    
def preprocess_modification(graph):
    model, preprocess = clip.load("ViT-L/14")
    for i in tqdm(range(graph.vert_count)):
        for j, img_bytes in enumerate(graph._images[i]):
            # Load image
            img = Image.open(io.BytesIO(img_bytes))
            preprocessed_image_tensor = inverse_normalize()(preprocess(img).unsqueeze(0))

            img_reversed = to_pil_image(preprocessed_image_tensor.squeeze(0).cpu())
            # Update the graph's _images list with the modified image
            with io.BytesIO() as output:
                img_reversed.save(output, format='PNG')
                graph._images[i][j] = output.getvalue()
    return graph

def recity_preprocess_modification(graph):
    model, preprocess = clip.load("ViT-L/14")
    for i in tqdm(range(graph.vert_count)):
        for j, img_bytes in enumerate(graph._images[i]):
            # Load image
            rectified_cropped_image = rectify_and_crop(np.array(PIL.Image.open(io.BytesIO(img_bytes))))
            preprocessed_image_tensor = inverse_normalize()(preprocess(rectified_cropped_image).unsqueeze(0))

            img_reversed = to_pil_image(preprocessed_image_tensor.squeeze(0).cpu())
            # Update the graph's _images list with the modified image
            with io.BytesIO() as output:
                img_reversed.save(output, format='PNG')
                graph._images[i][j] = output.getvalue()
    return graph

In [4]:
graph_recitified_cropped = recity_crop_modification(graph)

100%|██████████| 241/241 [00:13<00:00, 17.30it/s]


In [85]:
save_images(graph_recitified_cropped, "graph_recitified_cropped_images")

In [5]:
graph_recitified_cropped.save_to_file("graphs/small_graph_rectified_cropped.pkl")

#  CLassification of 278 Nodes

In [159]:
landmarks=['a glass building', 
            'a square with a large tree', 
            'a square with a tree', 
            'a white building', 
            'traffic lights', 
            'a white car', 
            'a disabled Parking spot', 
            'a trailer', 
            'a building with a red-black wall', 
            'a fire hydrant', 
            'a stop sign', 
            'an orange traffic cone', 
            'a manhole cover', 
            'a blue semi-truck', 
            'a red building', 
            'a picnic bench', 
            'a white truck', 
            'a white trailer', 
            'a traffic cone', 
            'a grove', 
            'a blue dumpster',
            'some thing else']

In [160]:
from lm_nav.optimal_route import nodes_landmarks_similarity

similarities = nodes_landmarks_similarity(graph_recitified_cropped, landmarks)


No Recitification


In [ ]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

# Apply softmax to convert logits to probabilities
probabilities = softmax(similarities)

# Find the index of the highest probability in each row
highest_prob_indices = np.argmax(probabilities, axis=1)

# Assuming you have a list of class names
class_names = landmarks  # Replace ... with your actual class names

# Prepare a list to hold class names with their corresponding highest probabilities
classified_names_with_probs = []

for i, index in enumerate(highest_prob_indices):
    class_name = class_names[index]
    prob = probabilities[i, index]
    classified_names_with_probs.append((class_name, prob))

# `classified_names_with_probs` now contains tuples of class names and their probabilities
for name, prob in classified_names_with_probs:
    print(f"{name}: {prob:.4f}")


In [171]:
probabilities[[69,80], 10]

array([0.04646572, 0.04658297])

In [ ]:
# Number of top rows to select for each class
n = 10

# Function to get top n rows for each class
def get_top_n_rows_for_each_class(probabilities, class_names, n=5):
    top_n_rows = {}
    for i, class_name in enumerate(class_names):
        # Get the probabilities for the current class
        class_probs = probabilities[:, i]
        
        # Find the indices of the top n probabilities
        # Argsort returns indices that would sort the array, so we take the last n indices for highest probabilities
        top_n_indices = np.argsort(class_probs)[-n:]
        
        # Reverse to get highest first
        top_n_indices = top_n_indices[::-1]
        
        # Store the indices and their probabilities in the dictionary
        top_n_rows[class_name] = [(index, class_probs[index]) for index in top_n_indices]
        
    return top_n_rows

# Get the top n rows with the highest probabilities for each class
top_n_rows = get_top_n_rows_for_each_class(probabilities, class_names, n)

# Display the results
for class_name, rows in top_n_rows.items():
    print(f"\n{class_name}:")
    for index, prob in rows:
        print(f"Index: {index}, Probability: {prob:.4f}")

In [168]:
root_folder_path = "top_10_images"
source = "graph_recitified_cropped_images/raw"
import os
import shutil
# Ensure root folder exists
if not os.path.exists(root_folder_path):
    os.makedirs(root_folder_path)

# Iterate over the classes and their top rows
for class_name, rows in top_n_rows.items():
    # Create a subfolder for the class if it doesn't exist
    class_folder_path = os.path.join(root_folder_path, class_name)
    if not os.path.exists(class_folder_path):
        os.makedirs(class_folder_path)
    
    # Copy the top images for the current class based on the indices in `rows`
    for i, v in enumerate(rows):
        row_index, _ = v
        for j in range(2):
            image_name = f"image_{row_index}_{j}_raw"  # Update this pattern as needed
            source_path = os.path.join(source, image_name)  # Update with your images' current location
            destination_path = os.path.join(class_folder_path,  f"{i}_{image_name}.png")
            
            # Copy image to the class subfolder
            shutil.copy(source_path, destination_path)

    print(f"Copied images for {class_name} to {class_folder_path}")


Copied images for a glass building to top_10_images/a glass building
Copied images for a square with a large tree to top_10_images/a square with a large tree
Copied images for a square with a tree to top_10_images/a square with a tree
Copied images for a white building to top_10_images/a white building
Copied images for traffic lights to top_10_images/traffic lights
Copied images for a white car to top_10_images/a white car
Copied images for a disabled Parking spot to top_10_images/a disabled Parking spot
Copied images for a trailer to top_10_images/a trailer
Copied images for a building with a red-black wall to top_10_images/a building with a red-black wall
Copied images for a fire hydrant to top_10_images/a fire hydrant
Copied images for a stop sign to top_10_images/a stop sign
Copied images for an orange traffic cone to top_10_images/an orange traffic cone
Copied images for a manhole cover to top_10_images/a manhole cover
Copied images for a blue semi-truck to top_10_images/a blue s

# Graph Optimization

In [11]:
import torch
import copy
from embedding_optimizer import EmbeddingOptimizer
from helper import preprocess_image

In [12]:
def optimize_image(current_image_data, target_image_data, l2_dist_threshold, cosine_sim_threshold, model, preprocess, optimizer, device):
    current_image = preprocess_image(current_image_data, preprocess, device)
    target_image = preprocess_image(target_image_data, preprocess, device)

    target_image_emb = model.encode_image(target_image)

    optimized_image, _, _, _ = optimizer.optimize_embeddings(current_image, target_image_emb, l2_dist_threshold, cosine_sim_threshold)
    optimized_image_inv = inverse_normalize()(optimized_image)
    return optimized_image_inv

In [13]:
def optimize_img_txt(current_image_data, target_text_embedding, l2_dist_threshold, cosine_sim_threshold, model, preprocess, optimizer, device):
    current_image = preprocess_image(current_image_data, preprocess, device)
    target_text_embedding = target_text_embedding.unsqueeze(0)
    optimized_image, _, _, _ = optimizer.optimize_embeddings(current_image, target_text_embedding, l2_dist_threshold, cosine_sim_threshold)
    optimized_image_inv = inverse_normalize()(optimized_image)
    return optimized_image_inv

In [86]:
def fgsm_graph(graph, input_nodes, target_nodes, args, output_dir="graphs/test.pkl"):
    print(args)
    device = "cuda:1" if torch.cuda.is_available() else "cpu"
    # model, preprocess = clip.load("ViT-B/32", device=device)
    model, preprocess = clip.load("ViT-L/14", device=device)
    #model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k', device=device)
    # tokenizer = open_clip.get_tokenizer('ViT-H-14')

    optimizer = EmbeddingOptimizer(model, args["learning_rate"], device)

    if len(input_nodes) != len(target_nodes):
        print('Number of input vertices should be equal to target vertices')
        return 
    new_images = copy.deepcopy(graph._images)
    source_images = copy.deepcopy(graph._images)
    for i in tqdm(range(graph.vert_count)):
        for j, img_bytes in enumerate(source_images[i]):
            if i not in input_nodes:
                pass
                #Load image
                # current_image_data = copy.deepcopy(source_images[i][j])
                # recitified_current_image = rectify_and_crop(np.array(PIL.Image.open(io.BytesIO(current_image_data))))
                # img_reversed = recitified_current_image
                # # Update the graph's _images list with the modified image
                # with io.BytesIO() as output:
                #     img_reversed.save(output, format='PNG')
                #     new_images[i][j] = output.getvalue()
            else:
                # Load image
                inp_ind = i
                tar_ind = input_nodes.index(i)
                print(f'Converting {inp_ind} to {target_nodes[tar_ind]}')
                current_image_data = copy.deepcopy(source_images[inp_ind][j])
                target_image_data = copy.deepcopy(source_images[target_nodes[tar_ind]][j])

                already_recitified_current_image = PIL.Image.fromarray(np.array(PIL.Image.open(io.BytesIO(current_image_data))))
                already_recitified_target_image = PIL.Image.fromarray(np.array(PIL.Image.open(io.BytesIO(target_image_data))))
                preprocessed_image_tensor = optimize_image(already_recitified_current_image, already_recitified_target_image,
                                                            args["l2_dist_threshold"],
                                                            args["cosine_sim_threshold"],
                                                            model,
                                                            preprocess,
                                                            optimizer,
                                                            device)
                img_reversed = to_pil_image(preprocessed_image_tensor.squeeze(0).cpu())
                
                # Update the graph's _images list with the modified image
                with io.BytesIO() as output:
                    img_reversed.save(output, format='PNG')
                    new_images[i][j] = output.getvalue()
                graph._images = new_images
                print(f'Saving Graph: {output_dir}')
                save_images(graph, output_dir)
    
    graph._images = new_images
    return graph

In [23]:
def fgsm_graph_img_txt(graph, input_nodes, target_texts, args):
    print(args)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    # model, preprocess = clip.load("ViT-B/32", device=device)
    model, preprocess = clip.load("ViT-L/14", device=device)
    tokenizer = clip.tokenize
    #model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k', device=device)
    # tokenizer = open_clip.get_tokenizer('ViT-H-14')

    optimizer = EmbeddingOptimizer(model, args["learning_rate"], device)
    target_texts = ["A photo of " + desc for desc in target_texts]
    text_embeddings = optimizer.get_text_embeddings(target_texts, tokenizer)

    if len(input_nodes) != len(target_texts):
        print('Number of input vertices should be equal to target vertices')
        return 
    new_images = copy.deepcopy(graph._images)
    
    for i in tqdm(range(len(input_nodes))):
        for j, img_bytes in enumerate(graph._images[input_nodes[i]]):
            # Load image
            current_image_data = graph._images[input_nodes[i]][j]
            target_text_data = text_embeddings[i]

            recitified_current_image = PIL.Image.fromarray(np.array(PIL.Image.open(io.BytesIO(current_image_data))))
            preprocessed_image_tensor = optimize_img_txt(recitified_current_image, target_text_data,
                                                        args["l2_dist_threshold"],
                                                        args["cosine_sim_threshold"],
                                                        model,
                                                        preprocess,
                                                        optimizer,
                                                        device)
            img_reversed = to_pil_image(preprocessed_image_tensor.squeeze(0).cpu())
            # Update the graph's _images list with the modified image
            with io.BytesIO() as output:
                img_reversed.save(output, format='PNG')
                new_images[input_nodes[i]][j] = output.getvalue()
    
    graph._images = new_images
    return graph

In [43]:
input_nodes = [0, 10, 15, 18, 20, 21, 23, 24, 48, 52, 76, 73, 1, 2, 3, 4, 5, 55, 70, 71, 72, 56]
target_nodes = [170, 135, 135, 135, 135, 135, 135, 135, 135, 135, 135, 217, 135, 135, 135, 135, 135, 135, 135, 135, 135, 77]

print(len(input_nodes))
print(len(target_nodes))

22
22


# Graph 3

In [87]:
# input_nodes = [7, 8, 57, 65, 77, 257, 260, 0, 10, 12, 13, 15, 17, 18, 20, 21, 23, 24, 25, 43, 44, 45, 46, 52, 75, 76, 253, 254, 72, 256, 1, 67, 68, 255, 71, 74, 82, 83, 85, 86, 62, 95, 42]
# target_nodes = [132, 132, 132, 132, 132, 132, 132, 263, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 217, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 77]

input_nodes = [112, 204, 111, 114, 115, 118, 109, 113, 200, 202, 216, 217, 78, 108, 207, 79, 208, 219, 220, 80]
target_nodes = [174, 174, 77, 224, 224, 224, 138, 164, 164, 164, 164, 164, 19, 132, 132, 132, 132, 132, 132, 168]

In [97]:

inp = [24, 77, 111, 112, 204, 238, 111, 3, 33, 35, 67, 70, 103, 109, 138, 150, 181, 109, 4, 12, 13, 14, 15, 19, 36, 62, 63, 78, 85, 86, 87, 88, 113, 160, 184, 185, 186, 187, 217, 234, 239, 271, 272, 273, 78, 0, 5, 6, 7, 8, 9, 10, 11, 16, 17, 49, 50, 51, 57, 58, 59, 64, 65, 74, 75, 79, 81, 83, 89, 92, 95, 96, 97, 98, 101, 102, 104, 105, 108, 142, 145, 146, 147, 148, 151, 155, 156, 157, 158, 159, 161, 162, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 182, 207, 208, 220, 222, 224, 225, 226, 229, 230, 231, 232, 237, 241, 242, 243, 244, 245, 246, 247, 248, 254, 257, 259, 260, 261, 263, 264, 265, 266, 267, 269, 270, 80]
tar = [174, 174, 174, 174, 174, 174, 77, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 138, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 19, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 168]

In [99]:
inps = []
tars = []
for i, val in enumerate(inp):
    if i not in input_nodes:
        inps.append(inp[i])
        tars.append(tar[i])

In [101]:
print(len(inps), len(tars))

131 131


In [117]:
original_graph = NavigationGraph("graphs/graph_rectified_cropped.pkl")

In [133]:
copy_graph = copy.deepcopy(original_graph)

In [112]:
from lm_nav.optimal_route import nodes_landmarks_similarity
final_landmark = 'a stop sign'

target_node = 80


similarities = nodes_landmarks_similarity(copy_graph, [final_landmark])

similarities[80]



No Recitification


array([0.19539315])

In [126]:
best_match = np.argmax(similarities, axis=0)[0]
worst_match = np.argmin(similarities, axis=0)[0]
print(best_match, worst_match)

77 135


In [120]:
args={"learning_rate":0.9, "l2_dist_threshold": 16, "cosine_sim_threshold":.95}

In [121]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)
model, preprocess = clip.load("ViT-L/14", device=device)
tokenizer = clip.tokenize
#model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k', device=device)
# tokenizer = open_clip.get_tokenizer('ViT-H-14')

optimizer = EmbeddingOptimizer(model, args["learning_rate"], device)

In [122]:
new_images = copy.deepcopy(copy_graph._images)

In [134]:
target_images = copy_graph._images[target_node]
for j, img_bytes in enumerate(target_images):
    print(f'Converting {target_node}-{j} to {best_match}-{j}')
    target_image_data = copy.deepcopy(target_images[j])
    best_image_data = copy.deepcopy(copy_graph._images[best_match][j])

    already_recitified_current_image = PIL.Image.fromarray(np.array(PIL.Image.open(io.BytesIO(target_image_data))))
    already_recitified_target_image = PIL.Image.fromarray(np.array(PIL.Image.open(io.BytesIO(best_image_data))))
    preprocessed_image_tensor = optimize_image(already_recitified_current_image, already_recitified_target_image,
                                                args["l2_dist_threshold"],
                                                args["cosine_sim_threshold"],
                                                model,
                                                preprocess,
                                                optimizer,
                                                device)
    img_reversed = to_pil_image(preprocessed_image_tensor.squeeze(0).cpu())
    
    # Update the graph's _images list with the modified image
    with io.BytesIO() as output:
        img_reversed.save(output, format='PNG')
        new_images[target_node][j] = output.getvalue()
copy_graph._images = new_images

Converting 80-0 to 77-0


Iter: 903, L2 Dist: 15.6484, Cos Sim: 0.9756:   6%|▌         | 904/15000 [01:32<24:02,  9.77it/s]


Converting 80-1 to 77-1


Iter: 711, L2 Dist: 15.9688, Cos Sim: 0.9766:   5%|▍         | 712/15000 [01:12<24:17,  9.80it/s]


In [135]:
from lm_nav.optimal_route import nodes_landmarks_similarity
final_landmark = 'a stop sign'

target_node = 80


similarities = nodes_landmarks_similarity(copy_graph, [final_landmark])

similarities[80]


No Recitification


array([0.23154464])

In [139]:
target_threshold = similarities[target_node][0]
target_threshold

0.2315446436405182

In [146]:
threshold_const = .95

In [154]:
indexes = list(np.where(similarities >= target_threshold*threshold_const)[0])
indexes.remove(target_node)
indexes

[77, 112, 141, 204, 257]

In [156]:
new_images = copy.deepcopy(copy_graph._images)
source_images = copy.deepcopy(copy_graph._images)
for i in tqdm(indexes):
    for j, img_bytes in enumerate(source_images[i]):
        # Load image
        print(f'Converting {i}-{j} to {worst_match}-{j}')
        current_image_data = copy.deepcopy(source_images[i][j])
        target_image_data = copy.deepcopy(source_images[worst_match][j])

        already_recitified_current_image = PIL.Image.fromarray(np.array(PIL.Image.open(io.BytesIO(current_image_data))))
        already_recitified_target_image = PIL.Image.fromarray(np.array(PIL.Image.open(io.BytesIO(target_image_data))))
        preprocessed_image_tensor = optimize_image(already_recitified_current_image, already_recitified_target_image,
                                                    args["l2_dist_threshold"],
                                                    args["cosine_sim_threshold"],
                                                    model,
                                                    preprocess,
                                                    optimizer,
                                                    device)
        img_reversed = to_pil_image(preprocessed_image_tensor.squeeze(0).cpu())
        
        # Update the graph's _images list with the modified image
        with io.BytesIO() as output:
            img_reversed.save(output, format='PNG')
            new_images[i][j] = output.getvalue()
copy_graph._images = new_images

  0%|          | 0/5 [00:00<?, ?it/s]

Converting 77-0 to 135-0


Iter: 1304, L2 Dist: 15.9922, Cos Sim: 0.9756:   9%|▊         | 1305/15000 [02:26<25:33,  8.93it/s]


Converting 77-1 to 135-1


 20%|██        | 1/5 [03:08<12:32, 188.09s/it]

...

 80%|████████  | 4/5 [10:31<02:45, 165.69s/it]

Converting 257-0 to 135-0


Iter: 535, L2 Dist: 15.7656, Cos Sim: 0.9761:   4%|▎         | 536/15000 [00:58<26:13,  9.19it/s]


Converting 257-1 to 135-1


100%|██████████| 5/5 [12:14<00:00, 146.91s/it]


In [157]:
from lm_nav.optimal_route import nodes_landmarks_similarity
final_landmark = 'a stop sign'

target_node = 80


similarities = nodes_landmarks_similarity(copy_graph, [final_landmark])

similarities[80]

best_node = np.argmax(similarities, axis=0)

print(best_node)

No Recitification
[80]


In [158]:
copy_graph.save_to_file("graphs/80_stop_sign.pkl")

In [ ]:
graph215_to_80_2 = fgsm_graph(copy_graph, inps, tars, args={"learning_rate":0.9, "l2_dist_threshold": 16, "cosine_sim_threshold":.95}, output_dir="graphs/inx215to80_2_checkpoint.pkl")

In [106]:
save_images(graph215_to_80_2, 'graph215_to_80_2_images')

In [107]:
graph215_to_80_2.save_to_file('graphs/inx215_to_80_2.pkl')

In [94]:
from lm_nav.optimal_route import nodes_landmarks_similarity

result = nodes_landmarks_similarity(original_graph, ['a stop sign', 'a manhole cover', 'a disabled Parking spot', 'a red building'])

No Recitification


In [95]:
print(result[[204, 138, 184, 170]]) 
      
print(result[[111, 109, 78, 80]])

[[0.24573301 0.14573407 0.17311713 0.16245659]
 [0.20108801 0.24157122 0.16400416 0.13532601]
 [0.17445144 0.15703727 0.21779397 0.22063816]
 [0.17777005 0.13775055 0.17209499 0.28330189]]
[[0.20970213 0.16256876 0.15840393 0.15158841]
 [0.18543546 0.17167214 0.16377217 0.15012589]
 [0.17796052 0.12819639 0.18708698 0.19787045]
 [0.19539812 0.15858787 0.15753396 0.16267893]]


In [96]:
from lm_nav.optimal_route import nodes_landmarks_similarity

result = nodes_landmarks_similarity(graph215_to_80, ['a stop sign', 'a manhole cover', 'a disabled Parking spot', 'a red building'])
print(result[[204, 138, 184, 170]]) 
      
print(result[[111, 109, 78, 80]])

No Recitification
[[0.20158547 0.204667   0.20546383 0.16358009]
 [0.20108801 0.24157122 0.16400416 0.13532601]
 [0.17445144 0.15703727 0.21779397 0.22063816]
 [0.17777005 0.13775055 0.17209499 0.28330189]]
[[0.24101552 0.17456573 0.20991287 0.20537138]
 [0.17926875 0.23444425 0.1645685  0.13086107]
 [0.1947396  0.13834321 0.21415901 0.18742812]
 [0.18395506 0.16836089 0.2004053  0.20390473]]


In [74]:
print(result[[204, 138, 184, 170]]) 
      
print(result[[111, 109, 78, 80]])

[[0.23782617 0.16758601 0.17363518 0.1718775 ]
 [0.18607658 0.21070144 0.175823   0.15714756]
 [0.19446927 0.15427817 0.2063664  0.21113059]
 [0.18234348 0.15716581 0.18115307 0.23745567]]
[[0.21738133 0.19014567 0.17288697 0.19671348]
 [0.19572923 0.19102642 0.17323929 0.16324607]
 [0.17854953 0.16072389 0.19860899 0.18983901]
 [0.18971071 0.15699568 0.1624265  0.18021673]]


In [75]:
from lm_nav.optimal_route import nodes_landmarks_similarity

result2 = nodes_landmarks_similarity(graph215_to_80, ['a stop sign', 'a manhole cover', 'a disabled Parking spot', 'a red building'])

No Recitification


In [76]:
print(result2[[204, 138, 184, 170]]) 
      
print(result2[[111, 109, 78, 80]])

[[0.20045486 0.20780437 0.19079348 0.16749252]
 [0.20108801 0.24157122 0.16400416 0.13532601]
 [0.17445144 0.15703727 0.21779397 0.22063816]
 [0.17777005 0.13775055 0.17209499 0.28330189]]
[[0.24529368 0.18213829 0.21401833 0.2034739 ]
 [0.18173014 0.22842434 0.17214432 0.13466151]
 [0.19862068 0.14767142 0.21750203 0.1828877 ]
 [0.18641327 0.16268645 0.19881625 0.1974327 ]]


In [12]:
from lm_nav.optimal_route import nodes_landmarks_similarity

result = nodes_landmarks_similarity(graph6, ['a manhole cover', 'a stop sign'])

No Recitification


In [13]:
result[77]

array([0.24521694, 0.22287041])

In [ ]:
original_graph = NavigationGraph("graphs/graph.pkl")

In [34]:
graph7 = fgsm_graph_img_txt(graph, [77], ['a manhole cover'], args={"learning_rate":0.1, "l2_dist_threshold": 81, "cosine_sim_threshold":.85})

{'learning_rate': 0.1, 'l2_dist_threshold': 81, 'cosine_sim_threshold': 0.85}


100%|██████████| 1/1 [44:11<00:00, 2651.29s/it]


In [35]:
from lm_nav.optimal_route import nodes_landmarks_similarity

result = nodes_landmarks_similarity(graph7, ['a manhole cover', 'a stop sign'])

No Recitification


In [36]:
result[77]

array([0.22188571, 0.29796207])

In [28]:
from lm_nav.optimal_route import nodes_landmarks_similarity

result = nodes_landmarks_similarity(graph, ['a manhole cover', 'a stop sign'])

No Recitification


In [29]:
result[77]

array([0.18534227, 0.26567292])

In [32]:
save_images(graph6, 'graph6_images')

In [33]:
save_images(graph7, 'graph7_images')